In [1]:
from pathlib import Path
import _fixpath

DAY_DIR = Path("/media/ko/MAUI63/2025-08-maui-winter-survey/2025-08-12")
GPS_DIR = DAY_DIR / "gps"
CAMERA_DIR = DAY_DIR / "cameras"


In [ ]:
from collections import namedtuple
from datetime import datetime, timedelta
import pytz

GpsTriggerEvent = namedtuple("GpsTriggerEvent", ["time", "lat", "lon", "alt", "ignore", "line_idx"])

# Parse the GPS data:
gps_points = []
for fpath in sorted(GPS_DIR.glob("*.CAM")):
    # Parse the start time from the filename:
    # Read the files:
    with open(fpath, "r") as f:
        for line_idx, line in enumerate(f):
            parts = line.split(",")
            week = int(parts[0])
            start_time = datetime(1980, 1, 6, tzinfo=pytz.utc) + timedelta(weeks=week)
            milliseconds_since_start_of_week = float(parts[1])
            num_satellites = int(parts[7])
            ignore = False
            if num_satellites < 12:
                print(f"Ignoring GPS point with {num_satellites} satellites")
                ignore = True
                continue
            lat = float(parts[8])
            lon = float(parts[9])
            alt = float(parts[10])
            time = start_time + timedelta(milliseconds=milliseconds_since_start_of_week)
            gps_points.append(GpsTriggerEvent(time, lat, lon, alt, ignore, line_idx))


Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
Ignoring GPS point with 0 satellites
I

In [97]:
# Plot the GPS points interactively with hover info:
import plotly.graph_objects as go


def make_plot(gps_points):
    lats = [p.lat for p in gps_points if not p.ignore]
    lons = [p.lon for p in gps_points if not p.ignore]
    alts = [p.alt for p in gps_points if not p.ignore]

    fig = go.Figure(
        data=[
            go.Scatter3d(
                x=lons,
                y=lats,
                z=alts,
                mode="lines+markers",
                marker=dict(size=4),
                text=[f"Index: {i}\nAlt: {alts[i]}" for i in range(len(gps_points))],
                hoverinfo="text",
            )
        ]
    )
    fig.update_layout(
        scene=dict(xaxis_title="Longitude", yaxis_title="Latitude", zaxis_title="Altitude"),
        title="GPS Track",
        width=800,
        height=800,
        margin=dict(l=0, r=0, b=0, t=40),
    )
    fig.show()


make_plot(gps_points)

In [ ]:
assert False, "Don't manually run this!"

# Manually remove any early triggers etc.

indices_to_ignore = [0, 1, 2]
gps_points = [p for i, p in enumerate(gps_points) if i not in indices_to_ignore]
print(f"Total GPS points: {len(gps_points)}")

make_plot(gps_points)

Total GPS points: 8427


In [98]:
from myutils import get_a7rv_image_timestamp
from tqdm import tqdm

# Get all the images per camera:
ImageMeta = namedtuple("ImageMeta", ["path", "timestamp"])

NZDT = pytz.timezone("Pacific/Auckland")

cameras = {}
for camera_dir in CAMERA_DIR.iterdir():
    camera = camera_dir.name
    assert camera in ("r09", "r28", "l09", "l28")
    images = []
    for img_path in tqdm(list(sorted(camera_dir.rglob("*.JPG"))), desc=f"Camera {camera}"):
        timestamp = get_a7rv_image_timestamp(img_path)
        if timestamp.tzinfo is None:
            # Assume NZDT:
            timestamp = NZDT.localize(timestamp)
        images.append(ImageMeta(path=img_path, timestamp=timestamp))

    # Important to sort by timestamp - we assume this later!
    cameras[camera] = sorted(images, key=lambda c: c.timestamp)


Camera r28: 100%|██████████| 3621/3621 [00:25<00:00, 142.75it/s]


In [43]:
# # OK, figure out the offset between each camera's timestamp and the GPS timestamp. How? Well, it'd be nice if we could assume the first image and GPS point
# # were in sync, but we can't. So, let's treat it as an optimisation problem .

# from scipy.optimize import minimize_scalar


# camera_offsets = {}
# for cam, images in cameras.items():
#     gps_timestamps = sorted([gps.time.timestamp() for gps in gps_points])
#     image_timestamps = sorted([img.timestamp.timestamp() for img in images])

#     def cost(offset):
#         # Use sorted lists and a moving pointer for efficient closest search
#         image_idx = 0
#         n_images = len(image_timestamps)
#         total_error = 0
#         for gps in gps_points:
#             gps_time = gps.time.timestamp() - offset
#             # Advance image_idx while next image is closer
#             while image_idx + 1 < n_images and abs(image_timestamps[image_idx + 1] - gps_time) < abs(
#                 image_timestamps[image_idx] - gps_time
#             ):
#                 image_idx += 1
#             # Clamp to min 5 so outliers don't bother us too much
#             total_error += abs(image_timestamps[image_idx] - gps_time)
#         # print(offset, total_error)
#         return total_error

#     # Get an initial estimate:
#     mean_gps = sum(gps_timestamps) / len(gps_timestamps)
#     mean_image = sum(image_timestamps) / len(image_timestamps)
#     initial_offset =  mean_gps - mean_image
#     print(f"Camera {cam}: initial offset estimate: {initial_offset:.1f} seconds")
#     res = minimize_scalar(cost, bounds=(-7200, 7200), tol=0.001, method="bounded")
#     assert res.success
#     offset = res.x
#     print(f"Camera {cam} final offset: {offset:.1f} seconds")
#     camera_offsets[cam] = offset
# print(camera_offsets)


In [ ]:
# OK, figure out the offset between each camera's timestamp and the GPS timestamp. How? Well, it'd be nice if we could assume the first image and GPS point
# were in sync, but we can't. So, let's treat it as an optimisation problem .
import numpy as np


def get_runs(times, max_interval_s, min_run_length):
    runs = []
    run_start_idx = 0
    run_len = 0
    for idx in range(1, len(times)):
        t0 = times[idx - 1]
        t1 = times[idx]
        if (t1 - t0).total_seconds() <= max_interval_s:
            run_len += 1
        else:
            if run_len >= min_run_length:
                runs.append((run_start_idx, run_len))
            run_len = 0
            run_start_idx = idx
    return runs


max_interval_s = 10
min_run_length = 50

gps_runs = get_runs([p.time for p in gps_points], max_interval_s, min_run_length)
print(f"Found {len(gps_runs)} GPS runs")
camera_offsets = []
camera_runs = {k: get_runs([img.timestamp for img in v], max_interval_s, min_run_length) for k, v in cameras.items()}

print(f"Num GPS runs: {len(gps_runs)}")
print(f"Num camera runs: {[(k, len(v)) for k, v in camera_runs.items()]}")

manual_offset = 0  # Use this to correct for big gaps in GPS track where it failed etc.

for gps_run_idx, (gps_start_idx, gps_run_len) in enumerate(gps_runs):
    gps_run_start_time = gps_points[gps_start_idx].time
    # For each camera, see if this same run has the same number of images:
    offsets = {}
    for cam, image_runs in camera_runs.items():
        image_run_idx = gps_run_idx + manual_offset
        if not 0 <= image_run_idx < len(image_runs):
            continue
        image_run_start_idx, img_run_len = image_runs[image_run_idx]
        # Why -1 below? Not sure. Seems like each time we trigger a run there's one extra GPS trigger than actual frames.
        if abs(img_run_len - gps_run_len) > 2:
            print(f"Run {gps_run_idx} NOT matched for camera {cam} (GPS len={gps_run_len}, img len={img_run_len})")
            break
        else:
            # Now we have a run of images and a run of GPS points. Let's align them.
            img_run_start_time = cameras[cam][image_run_start_idx].timestamp
            offset = (gps_run_start_time - img_run_start_time).total_seconds()
            # print(f"Camera {cam}: Offset is {offset} (GPS time - Image time)")
            offsets[cam] = offset
    else:
        # print(f"Run {gps_run_idx} matched for all cameras!")
        camera_offsets.append(offsets)

# For each camera, print the offsets:
camera_average_offsets = {}
for cam in ("r09", "r28", "l09", "l28"):
    cam_offsets = [o[cam] for o in camera_offsets if cam in o]
    avg_offset = np.median(cam_offsets)
    camera_average_offsets[cam] = avg_offset
    print("-" * 100)
    print(cam)
    print(f"Camera {cam}: Average offset is {avg_offset:.3f} seconds over {len(cam_offsets)} runs")
    for i, offset in enumerate(cam_offsets):
        print(f"Run {i: 3d}: Offset is {offset:.3f} ({offset - avg_offset:+.3f} from avg)")
camera_average_offsets
del camera_offsets


Found 15 GPS runs
Num GPS runs: 15
Num camera runs: [('l09', 16), ('l28', 15), ('r09', 15), ('r28', 16)]
----------------------------------------------------------------------------------------------------
r09
Camera r09: Average offset is -3692.554 seconds over 15 runs
Run   0: Offset is -3692.553 (+0.001 from avg)
Run   1: Offset is -3690.757 (+1.797 from avg)
Run   2: Offset is -3692.555 (-0.001 from avg)
Run   3: Offset is -3690.753 (+1.801 from avg)
Run   4: Offset is -3692.554 (+0.000 from avg)
Run   5: Offset is -3694.153 (-1.599 from avg)
Run   6: Offset is -3692.556 (-0.002 from avg)
Run   7: Offset is -3694.553 (-1.999 from avg)
Run   8: Offset is -3692.513 (+0.041 from avg)
Run   9: Offset is -3694.952 (-2.398 from avg)
Run  10: Offset is -3690.715 (+1.839 from avg)
Run  11: Offset is -3693.713 (-1.159 from avg)
Run  12: Offset is -3692.115 (+0.439 from avg)
Run  13: Offset is -3694.113 (-1.559 from avg)
Run  14: Offset is -3692.115 (+0.439 from avg)
------------------------

In [100]:
# Cool, now plot the aligned data on a graph. Make it interactive so we can zoom in to parts of it.
import plotly.graph_objects as go


def plot_aligned_data(gps_points, cameras, camera_offsets):
    fig = go.Figure()

    timestamps = [p.time.astimezone(NZDT) for p in gps_points]
    fig.add_trace(
        go.Scatter(
            x=timestamps,
            y=[1] * len(timestamps),
            mode="markers",
            name="GPS Points",
            marker=dict(size=4, color="blue"),
        )
    )

    # Plot camera points:
    colors = {"r09": "red", "r28": "green", "l09": "orange", "l28": "purple"}
    for idx, (cam, images) in enumerate(cameras.items()):
        offset = timedelta(seconds=camera_average_offsets[cam])
        timestamps = [img.timestamp + offset for img in images]
        fig.add_trace(
            go.Scatter(
                x=timestamps,
                y=[1 + (idx + 1) / 10] * len(timestamps),
                mode="markers",
                name=cam,
                marker=dict(size=2, color=colors[cam]),
            )
        )

    fig.show()


plot_aligned_data(gps_points, cameras, camera_average_offsets)

In [ ]:
max_time_diff_s = 0.75  # This is fine tuned somewhat
assert max_time_diff_s < 1.7 / 2, (
    "the time between points got down to 1.7s with a headwind, so if you go below half that, you'll match in both directions"
)

# Precompute sorted timestamps for each camera
sorted_camera_timestamps = {
    cam: [img.timestamp + timedelta(seconds=camera_average_offsets[cam]) for img in images]
    for cam, images in cameras.items()
}
matched_images = {cam: set() for cam in cameras.keys()}
output = []
camera_sorted_idxs = {cam: 0 for cam in cameras.keys()}

for gps_point in tqdm(gps_points):
    gps_time = gps_point.time
    matched = {}
    for cam, idx in camera_sorted_idxs.items():
        timestamps = sorted_camera_timestamps[cam]
        while True:
            if idx >= len(timestamps):
                break
            t = timestamps[idx]
            if (t - gps_time).total_seconds() < -max_time_diff_s:
                # Too early, discard this timestamp
                idx += 1
            elif (t - gps_time).total_seconds() > max_time_diff_s:
                # Gone too far
                break
            else:
                assert abs((t - gps_time).total_seconds()) <= max_time_diff_s
                matched[cam] = (t, cameras[cam][idx])
                idx += 1
                break
        camera_sorted_idxs[cam] = idx
    # print(camera_sorted_idxs)

    output.append((gps_point, matched))
output[0]

100%|██████████| 3639/3639 [00:00<00:00, 88162.15it/s]


(GpsTriggerEvent(time=datetime.datetime(2025, 8, 15, 19, 59, 54, 667000, tzinfo=<UTC>), lat=-36.831254071, lon=174.514141751, alt=712.066, ignore=False),
 {})

In [104]:
def plot_aligned_data(matched, gps_points, cameras, camera_offsets):
    fig = go.Figure()

    offsets = {"l09": 0.1, "l28": 0.2, "r09": 0.3, "r28": 0.4}

    # Plot unmatched gps:
    matched_gps_times = [p.time.astimezone(NZDT) for p, cam_dict in matched if not cam_dict]
    fig.add_trace(
        go.Scatter(
            x=matched_gps_times,
            y=[1] * len(matched_gps_times),
            mode="markers",
            name="GPS Point",
            marker=dict(size=8, color="red"),
            opacity=0.5,
        )
    )

    # And unmatched camera points:
    for cam in cameras.keys():
        all_ts = set(sorted_camera_timestamps[cam])
        matched_ts = []
        for gps_point, cam_dict in matched:
            if len(cam_dict) == 0 or cam not in cam_dict:
                continue
            matched_ts.append(cam_dict[cam][0])
        unmatched_ts = sorted(all_ts - set(matched_ts))
        ts = [t.astimezone(NZDT) for t in unmatched_ts]
        fig.add_trace(
            go.Scatter(
                x=ts,
                y=[1 + offsets[cam]] * len(ts),
                mode="markers",
                name=cam,
                marker=dict(size=8, color="red"),
                opacity=0.5,
            )
        )

    timestamps = [p.time.astimezone(NZDT) for p in gps_points]
    fig.add_trace(
        go.Scatter(
            x=timestamps,
            y=[1] * len(timestamps),
            mode="markers",
            name="GPS Points",
            marker=dict(size=4, color="blue"),
        )
    )

    # Plot camera points:
    colors = {"r09": "red", "r28": "green", "l09": "orange", "l28": "purple"}
    for idx, (cam, images) in enumerate(cameras.items()):
        offset = timedelta(seconds=camera_offsets[cam])
        timestamps = [img.timestamp + offset for img in images]
        fig.add_trace(
            go.Scatter(
                x=timestamps,
                y=[1 + offsets[cam]] * len(timestamps),
                mode="markers",
                name=cam,
                marker=dict(size=2, color=colors[cam]),
            )
        )

    fig.show()


plot_aligned_data(output, gps_points, cameras, camera_average_offsets)

In [105]:
# What are the counts of frames per GPS point?
from collections import Counter

counts = Counter()
for gps_point, cam_dict in output:
    counts[len(cam_dict)] += 1
print(counts)

Counter({4: 3609, 0: 16, 3: 9, 1: 4, 2: 1})


In [106]:
import json
import constants

# Phew, we're done! Let's create an output json:
json_output = {
    "cameras": list(cameras.keys()),
    "frames": [],
}
for gps_point, cam_dict in output:
    entry = {
        "lat": gps_point.lat,
        "lon": gps_point.lon,
        "alt": gps_point.alt,
        "gps_time": gps_point.time.isoformat(),
        "images": {
            cam: dict(path=str(img_meta.path.relative_to(DAY_DIR)), t=t.isoformat())
            for cam, (t, img_meta) in cam_dict.items()
        },
    }
    json_output["frames"].append(entry)
opath = constants.DATA_DIR / "processed_frames.json"
with open(opath, "w") as f:
    json.dump(json_output, f, indent=2)
print(f"Move {opath} to {DAY_DIR / 'processed_frames.json'} as we've mounted it read-only")

Move /workspaces/cv/data/processed_frames.json to /media/ko/MAUI63/2025-08-maui-winter-survey/2025-08-16/processed_frames.json as we've mounted it read-only
